In [113]:
import mfem.ser as mfem
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

### Вариант 13

## Сильная и слабая формы задачи

### Сильная формулировка
### Задача теплопроводности в стационарной форме

Уравнение теплопроводности:

$$
-\nabla \cdot (k(x) \nabla T(x)) = q(x), \quad x \in \Omega
$$

где:
- $T(x)$ — температура в точке \( x \), зависящая от координат;
- $k(x)$ — коэффициент теплопроводности, который зависит от материала (воздух или алюминий);
- $q(x)$ — источник тепла.

### Граничные условия

1. **Температура на поверхности окна :**
   $$ T(x) |_{\partial \Omega_D} = -10.0, \quad x \in \partial \Omega_D $$
   Температура на границе, соответствующей окну, фиксирована и равна -10.0°C.

2. **Граничное условие Неймана на стенах:**
   $$ \frac{\partial T(x)}{\partial n} |_{\partial \Omega_N} = 0, \quad x \in \partial \Omega_N $$
   Поток тепла через стены отсутствует.

### Свойства материалов

1. **Коэффициент теплопроводности $k(x)$:**
   $$ k(x) =
   \begin{cases}
      0.022, & \text{если } x \in \text{воздух}, \\
      209.3, & \text{если } x \in \text{алюминий (радиатор)}.
   \end{cases}
   $$

2. **Источник тепла $q(x)$:**
   $$ q(x) =
   \begin{cases}
      0, & \text{если } x \in \text{воздух}, \\
      heat, & \text{если } x \in \text{алюминий (радиатор)}.
   \end{cases}
   $$


### Интерпретация задачи

Задача описывает теплопроводность в комнате с радиатором, сделанным из алюминия, который выделяет тепло. Граничные условия задают температуру на окне и отсутствие теплового потока на стенах. Важно учитывать различные коэффициенты теплопроводности для воздуха и алюминия, а также источник тепла - радиатор. 

### Слабая формулировка 

Найти $$ T \in V_D = \{f \in H^1(\Omega): f|_{\partial \Omega_D} = -10\}$$
Удовлетворяющую уравнению 
\
$$\int_{\Omega} k \nabla T \cdot \nabla \upsilon dV = \int_{\Omega} q \upsilon dV, \forall \upsilon \in V_0 = \{f \in H^1(\Omega): f|_{\partial \Omega_D} = 0\}$$

## Построение модели: FreeCAD + GMSH
### Построены 3 разные сетки

In [138]:
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "Screenshot%20from%202024-11-18%2018-10-53.png", width=800, height=800)

In [139]:
Image(url= "Screenshot%20from%202024-11-18%2018-12-14.png", width=800, height=800)

In [140]:
Image(url= "Screenshot%20from%202024-11-18%2018-14-26.png", width=800, height=800)

## Код

In [152]:
Image(url= "9d8e942s-960 (1).jpg", width=500, height=500)

### Получение решения

In [115]:
def solve(mesh, fespace, heat):
    """
    Решение задачи теплопередачи методом конечных элементов (FEM) с учетом граничных условий Дирихле и Неймана.
    
    Эта функция решает задачу теплопередачи в области, с учетом различных теплопроводностей для разных материалов.
    Используются граничные условия:
    - Дирихле: температура на окне и стенах.
    - Нейман: отсутствие потока тепла через стены.

    :param mesh: **mfem.Mesh** — сетка, определяющая геометрию области. 
    
    :param fespace: **mfem.FiniteElementSpace** — пространство конечных элементов. Это пространство, в котором будет решаться задача теплопередачи.
    
    :param heat: **float** — значение источника тепла, который применяется в радиаторе (алюминиевый радиатор). Это значение определяет мощность источника тепла, добавляемую в соответствующую часть области.

    :return: **mfem.GridFunction** — решение задачи в виде функции сетки, где значения представляют температуры в узлах сетки.
    """
    
    t1 = mfem.ConstantCoefficient(-10.0) # Температура на окне 
    t2 = mfem.ConstantCoefficient(0) 
    
    ess_bdr1 = mfem.intArray([0]*mesh.bdr_attributes.Size()) 
    ess_bdr1[0] = 1 # Окно 
    
    # Граничные условия Неймана для всех стен  - отсуствие потока тепла через стенки
    ess_bdr2 = mfem.intArray([0]*mesh.bdr_attributes.Size())
    ess_bdr2[1] = 1 # Стенки
    
    # Применение граничных условий Дирихле
    ess_tdof_list = mfem.intArray()
    ess_bdr = mfem.intArray([1]*mesh.bdr_attributes.Size())
    ess_bdr[1] = 0
    fespace.GetEssentialTrueDofs(ess_bdr, ess_tdof_list)
        
    
    # Теплопроводности для разных областей
    k_air = 0.022  # Теплопроводность для воздуха
    k_metal = 209.3 # Теплопроводность для радиатора (алюминий)
    
    # Tеплопроводности для разных областей
    k_values = mfem.Vector(mesh.attributes.Max())
    k_values[0] = k_air  # Воздух в комнате
    k_values[1] = k_metal  # Радиатор
    
    # Создаем коэффициент теплопроводности 
    k_pw_const = mfem.PWConstCoefficient(k_values)

    class VolumeSource(mfem.PyCoefficient):
        def __init__(self, mesh):
            super().__init__()
            self.mesh = mesh

        def Eval(self, Ttr, ip):
            element_attribute = Ttr.ElementNo

            if self.mesh.GetAttribute(element_attribute) == 2:
                return heat
            elif self.mesh.GetAttribute(element_attribute) == 1:
                return 0.0
            else:
                return 0.0

    rhs = VolumeSource(mesh)

    a = mfem.BilinearForm(fespace)
    a.AddDomainIntegrator(mfem.DiffusionIntegrator(k_pw_const))
    a.Assemble()
    
    # Создание линейной формы для правой части
    b = mfem.LinearForm(fespace)
    b.AddDomainIntegrator(mfem.DomainLFIntegrator(rhs))
    b.AddDomainIntegrator(mfem.BoundaryLFIntegrator(t2), ess_bdr2)
    b.Assemble()
    
    # Задание граничных условий Дирихле
    x = mfem.GridFunction(fespace)
    x.Assign(0.0) # Начальное значение температуры во всей области
    x.ProjectBdrCoefficient(t1, ess_bdr1)
    
    # Формирование линейной системы (AX=B)
    A = mfem.OperatorPtr()
    B = mfem.Vector()
    X = mfem.Vector()
    a.FormLinearSystem(ess_tdof_list, x, b, A, X, B)
    
    AA = mfem.OperatorHandle2SparseMatrix(A)
    M = mfem.GSSmoother(AA)
    mfem.PCG(AA, M, B, X, 1, 500, 1e-12, 0.0)
    a.RecoverFEMSolution(X, b, x)

    return x

In [116]:
def save_file(mesh, x, file_name):
    
    pd = mfem.ParaViewDataCollection(file_name, mesh)
    pd.SetPrefixPath("ParaView")
    pd.RegisterField("temp", x)
    pd.Save()

In [117]:
def find_closest_vertex(verts, point):
    
    distances = cdist([point], verts)  
    index = np.argmin(distances)
    return index

### Расчет нужной мощности радиатора

In [118]:
def find_f_newton(mesh, fespace, centre, temp_goal, heat_guess, tolerance, epsilon=1e-3, max_iterations=100):
    """
    Метод Ньютона для нахождения значения f, при котором температура в точке centre будет близка к temp_goal.

    :param mesh: Сетка.
    :param fespace: Пространство конечных элементов.
    :param centre: Точка, в которой нужно достичь целевой температуры.
    :param temp_goal: Целевая температура.
    :param heat_guess: Начальная догадка для значения параметра источника тепла.
    :param tolerance: Точность, с которой должна быть достигнута целевая температура.
    :param epsilon: Малое число для вычисления производной (по умолчанию 1e-3).
    :param max_iterations: Максимальное количество итераций для метода Ньютона.
    
    :return: Решение x, температура в точке, найденное значение мощности, координаты вершины
    """

    heat_source = heat_guess  
    iteration = 0
    verts = mesh.GetVertexArray()

    # Найдем индекс ближайшей вершины к заданной точке (centre)
    index_nearest_centre = find_closest_vertex(verts, centre)

    # Итерации метода Ньютона
    while iteration < max_iterations:
        # Решаем задачу методом конечных элементов для текущего значения f
        x = solve(mesh, fespace, heat_source)
        sol = x.GetDataArray()

        # Получаем текущую температуру в ближайшей вершине
        current_temp = sol[index_nearest_centre]
        print("Temperature: ", current_temp)
        # Вычисляем ошибку
        error = current_temp - temp_goal

        # Если ошибка достаточно мала, выходим из цикла
        if abs(error) < tolerance:
            break

        x1 = solve(mesh, fespace, heat_source + epsilon)
        sol1 = x1.GetDataArray()
        temp1 = sol1[index_nearest_centre]
        derivative = (temp1 - current_temp) / epsilon

        # Обновляем значение heat_source с использованием формулы Ньютона
        heat_source_next = heat_source - error / derivative
        heat_source = heat_source_next
        iteration += 1

    return x, current_temp, heat_source, verts[index_nearest_centre]

### Расчет максимальной температуры радиатора

In [119]:
def find_max_radiator_temp(radiator_position, mesh, x):
    """
    Функция находит максимальную температуру в радиаторе, которая определяется
    температурой вершин, попадающих в заданную область (позицией радиатора) на сетке.

    Аргументы:
    radiator_position (tuple): ((min_x, min_y, min_z), (max_x, max_y, max_z)).
    mesh (object): Объект сетки (mesh), содержащий информацию о вершинах.
    sol (list): Список или массив температур, соответствующих каждой вершине сетки.

    Возвращаемое значение:
    float: Максимальная температура среди вершин, попадающих в заданную область.
    """
    
    verts = mesh.GetVertexArray()  
    sol = x.GetDataArray()
    temp_radiator_max = 0 

    for i, vertex in enumerate(verts):
        if all(radiator_position[0][j] <= vertex[j] <= radiator_position[1][j] for j in range(0, 3)):
            temp_radiator_max = max(temp_radiator_max, sol[i])

    return temp_radiator_max

In [120]:
def calculate_for_mesh(meshname, filename):
    
    temp_goal = 19.0
    centre_coord = np.array([2350, 2550, 1550])
    radiator_position = ((1800, 0, 200),(2900, 150, 800))
    heat_guess = 1e-6
    tolerance = 1e-3
    
    mesh = mfem.Mesh(meshname, 1, 1) 
    fec = mfem.H1_FECollection(1, mesh.Dimension())   
    fespace = mfem.FiniteElementSpace(mesh, fec)
    x, current_temp, heat_source, verts = find_f_newton(mesh, fespace, centre_coord, temp_goal, heat_guess, tolerance)
    save_file(mesh, x, filename)
    
    vertex = f"({verts[0]:.0f}, {verts[1]:.0f}, {verts[2]:.0f})"
    max_temp = find_max_radiator_temp(radiator_position, mesh, x)
    return [fespace.GetTrueVSize(), current_temp, heat_source, vertex, max_temp]

### Вычисления и результаты

In [121]:
sol1 = calculate_for_mesh('mesh1_', 'mesh1')
sol2 = calculate_for_mesh('mesh2_', 'mesh2')
sol3 = calculate_for_mesh('mesh3_', 'mesh3')

Average reduction factor = 0.766175
   Iteration :   0  (B r, r) = 12459.7
   Iteration :   1  (B r, r) = 1984.28
Temperature:  -9.51222709423543
   Iteration :   2  (B r, r) = 740.629
   Iteration :   3  (B r, r) = 392.772
   Iteration :   4  (B r, r) = 245.013
   Iteration :   5  (B r, r) = 171.736
   Iteration :   6  (B r, r) = 131.635
   Iteration :   7  (B r, r) = 113.723
   Iteration :   8  (B r, r) = 103.061
   Iteration :   9  (B r, r) = 96.5039
   Iteration :  10  (B r, r) = 87.8264
   Iteration :  11  (B r, r) = 74.0759
   Iteration :  12  (B r, r) = 62.7986
   Iteration :  13  (B r, r) = 50.5698
   Iteration :  14  (B r, r) = 44.7376
   Iteration :  15  (B r, r) = 38.5191
   Iteration :  16  (B r, r) = 36.7871
   Iteration :  17  (B r, r) = 33.2997
   Iteration :  18  (B r, r) = 34.7346
   Iteration :  19  (B r, r) = 47.8148
   Iteration :  20  (B r, r) = 73.3512
   Iteration :  21  (B r, r) = 94.9521
   Iteration :  22  (B r, r) = 98.6307
   Iteration :  23  (B r, r) = 79.3

In [141]:
df = pd.DataFrame([{"True size": sol1[0], "Center temperature": sol1[1], "Power": sol1[2], "Vertex": sol1[3], "Radiator max temperature": sol1[4]}])
df.loc[len(df.index)] = sol2
df.loc[len(df.index)] = sol3
pd.set_option("display.precision", 6)
df

,True size,Center temperature,Power,Vertex,Radiator max temperature
0,9500,19.0,0.000059,"(2454, 2528, 1513)",78.795069
1,4651,19.0,0.000065,"(2171, 2483, 1537)",83.614475
2,1563,19.0,0.000075,"(2330, 2677, 1821)",83.097348


## Просмотр результатов в ParaView

In [146]:
Image(url= "Screenshot from 2024-11-18 18-42-43.png", width=1000, height=1000)

In [145]:
Image(url= "Screenshot from 2024-11-18 18-43-52.png", width=1000, height=1000)

In [147]:
Image(url= "Screenshot from 2024-11-18 18-43-22.png", width=1000, height=1000)

In [148]:
Image(url= "Screenshot from 2024-11-18 18-44-36.png", width=1000, height=1000)